In [38]:
from __future__ import absolute_import, division, print_function
from builtins import (
    ascii, bytes, chr, dict, filter, hex, input, int, map,
    next, oct, open, pow, range, round, str, super, zip)

import os
from itertools import cycle
from functools import partial
import random
from math import pi

import numpy as np
from sklearn.cluster import KMeans

# Imports for working with shapefiles
import pyproj
from shapely.geometry import shape, MultiPolygon
from shapely.ops import transform
from descartes import PolygonPatch
import fiona
from fiona.crs import from_epsg

In [3]:
wisc_census_blocks = 'data/block_groups/shapes/tl_2013_55_bg.shp'

In [4]:
project = partial(
    pyproj.transform,
    pyproj.Proj(init='epsg:4326'), 
    pyproj.Proj(init='epsg:3695')
)

In [41]:
with fiona.open(wisc_census_blocks) as f:
    blocks = [
        {
            'shape': transform(project, shape(block['geometry'])), 
             'geoid': block['properties']['GEOID']
        }
        for block in f
    ]
for block in blocks:
    block['centroid'] = np.array([coord[0] for coord in block['shape'].centroid.xy])

In [46]:
coord_array = np.array([block['centroid'] for block in blocks])

In [53]:
model = KMeans(n_clusters=8, max_iter=1000, n_jobs=-1)

In [54]:
labels = model.fit_predict(coord_array)

In [59]:
for label, block in zip(labels, blocks):
    block['label'] = int(label)

In [60]:
type(blocks[0]['label'])

future.types.newint.newint